In this notebook, I want to compute the uncertainty of the equilibrium lattice constant and the cohesive energy from the bootstrap ensembles.

In [1]:
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import sys

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use("default")

WORK_DIR = Path().absolute()
sys.path.append(str(WORK_DIR.parent))

In [2]:
from energyvslatconst.relaxation_latconst import equilibrate_graphene

In [3]:
# Read setting file
ROOT_DIR = WORK_DIR.parent
with open(ROOT_DIR / "settings.json", "r") as f:
    settings = json.load(f)
partition = settings["partition"]
RES_DIR = WORK_DIR / "results" / f"{partition}_partition"
if not RES_DIR.exists():
    RES_DIR.mkdir(parents=True)

In [4]:
a0_e0_file = RES_DIR / "uncertainty_latconst_ecoh_graphene.npz"
if a0_e0_file.exists():
    a0_e0 = np.load(a0_e0_file)
    a0_list = a0_e0["a0_list"]
    e0_list = a0_e0["e0_list"]
else:
    a0_list = []
    e0_list = []

    ainit = 2.466  # From materials project for graphite-like structure
    for ii in tqdm(range(100)):
        # Equilibration
        potential = f"DUNN_C_randinit_{ii:03d}"
        a0, e0 = equilibrate_graphene(potential, ainit)
        a0_list = np.append(a0_list, a0)
        e0_list = np.append(e0_list, e0)
    
    np.savez(a0_e0_file, a0_list=a0_list, e0_list=e0_list)

In [5]:
# Lattice constant
print("Lattice constant")
print("Mean:", np.mean(a0_list))
print("Stdev:", np.std(a0_list))

Lattice constant
Mean: 2.4610090471875004
Stdev: 0.005542758026657445


In [6]:
# Cobesive energy
print("Cohesive energy")
print("Mean:", np.mean(e0_list))
print("Stdev:", np.std(e0_list))

Cohesive energy
Mean: -8.07037674881962
Stdev: 0.007323655317787101
